In [2]:
import json
import re
import random
import tqdm

In [75]:
data_splits = ["test", "train", "val"]
for data_split in data_splits:
    with open(f"{data_split}.txt", "r") as f, open(f"{data_split}.trace", "r") as t, open(f"{data_split}.jsonl", "w") as json_f:
        data = f.read()
        traces = t.readlines()
        data = data.split("\t1\n")
        data = data[:-1]
        
        for i in tqdm.tqdm(range(len(data))):
            chunk = data[i]
            trace = traces[i]
            storyType, questionType = trace.split(",")[-1].strip(), trace.split(",")[-2].strip()
            story, question, label = "\n".join(chunk.split("\t")[0].strip().split("\n")[:-1]), chunk.split("\t")[0].strip().split("\n")[-1], chunk.split("\t")[1].strip()
            question = question.split(" ", 1)[1]
            
            try:
                # Find subject of thing
                pattern = r'\bthe\b\s+(\b[\w-]+\b)'
                subject = re.findall(pattern, question, re.IGNORECASE)[0]
                # Find containers
                pattern = fr'\b{subject} is in the\b\s+(\b[\w-]+\b)'
                container1 = re.findall(pattern, story, re.IGNORECASE)[0]
                pattern = fr'\b{subject} to the\b\s+(\b[\w-]+\b)'
                container2 = re.findall(pattern, story, re.IGNORECASE)[0]
                containers = [container1, container2]
                random.shuffle(containers)
            except:
                break
            
            
            jsonData = {
                            "story": story,
                            "question": question,
                            "answer": label,
                            "story_type": storyType,
                            "question_type": questionType,
                            "containers" : containers
                        }
            
            json_f.write(json.dumps(jsonData)+"\n")

100%|██████████| 6004/6004 [00:00<00:00, 43525.21it/s]


In [8]:
# Balanced split
data_splits = ["test"]
maxPerCat = 100

for data_split in data_splits:
    categories = {}
    with open(f"{data_split}.txt", "r") as f, open(f"{data_split}.trace", "r") as t, open(f"{data_split}_balanced.jsonl", "w") as json_f:
        data = f.read()
        traces = t.readlines()
        data = data.split("\t1\n")
        data = data[:-1]
        
        for i in tqdm.tqdm(range(len(data))):
            chunk = data[i]
            trace = traces[i]
            storyType, questionType = trace.split(",")[-1].strip(), trace.split(",")[-2].strip()
            story, question, label = "\n".join(chunk.split("\t")[0].strip().split("\n")[:-1]), chunk.split("\t")[0].strip().split("\n")[-1], chunk.split("\t")[1].strip()
            question = question.split(" ", 1)[1]
            
            try:
                # Find subject of thing
                pattern = r'\bthe\b\s+(\b[\w-]+\b)'
                subject = re.findall(pattern, question, re.IGNORECASE)[0]
                # Find containers
                pattern = fr'\b{subject} is in the\b\s+(\b[\w-]+\b)'
                container1 = re.findall(pattern, story, re.IGNORECASE)[0]
                pattern = fr'\b{subject} to the\b\s+(\b[\w-]+\b)'
                container2 = re.findall(pattern, story, re.IGNORECASE)[0]
                containers = [container1, container2]
                random.shuffle(containers)
            except:
                break
            
            
            jsonData = {
                            "story": story,
                            "question": question,
                            "answer": label,
                            "story_type": storyType,
                            "question_type": questionType,
                            "containers" : containers
                        }
            
            if categories.get(questionType, 0) >= maxPerCat:
                continue
            
            categories[questionType] = categories.get(questionType, 0) + 1
            
            
            json_f.write(json.dumps(jsonData)+"\n")
        
print(categories)

  0%|          | 0/6004 [00:00<?, ?it/s]

100%|██████████| 6004/6004 [00:00<00:00, 53800.81it/s]

{'second_order_0_no_tom': 100, 'first_order_1_no_tom': 100, 'reality': 100, 'first_order_0_no_tom': 100, 'first_order_1_tom': 100, 'second_order_1_tom': 100, 'second_order_1_no_tom': 100, 'first_order_0_tom': 100, 'memory': 100, 'second_order_0_tom': 100}


In [9]:
def shuffle_jsonl(file_path):
    # Read data
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Decode JSON objects
    data = [json.loads(line) for line in lines]

    # Shuffle data
    random.shuffle(data)

    # Encode JSON objects
    shuffled_lines = [json.dumps(obj) + '\n' for obj in data]

    # Write data
    with open(file_path, 'w') as f:
        f.writelines(shuffled_lines)
        
shuffle_jsonl("test_balanced.jsonl")